In [1]:
import os
import gc
import warnings
warnings.filterwarnings("ignore")

In [2]:
import evaluate
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from datasets import Dataset, DatasetDict
from transformers import logging, AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import PeftConfig, PeftModel
tqdm.pandas()
logging.set_verbosity_error()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gc.collect()
torch.manual_seed(42)

bin d:\PythonVenv\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [ ]:
train_data = pd.read_csv('../dataset/full_train_data_summarization.csv')
train_data_with_title = pd.read_csv('../dataset/full_train_data_title_summarization.csv')
test_data = pd.read_csv('../dataset/full_test_data_summarization.csv')

In [3]:
model_name = 'VietAI/vit5-base'

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
def preprocess_function(examples):
  inputs = [doc for doc in examples["context"]]
  model_inputs = tokenizer(inputs, max_length=2048, truncation=True, padding=True)
  labels = tokenizer(text_target=examples["summarization"], max_length=768, truncation=True, padding=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [7]:
new_data = DatasetDict({
    "train": Dataset.from_dict(train_data),
    "test": Dataset.from_dict(test_data)
})

In [ ]:
# Map dataset with multiprocessing
tokenized_new_data = new_data.map(preprocess_function, batched=True, num_proc=8)

In [ ]:
# Map dataset not with multiprocessing
tokenized_new_data = new_data.map(preprocess_function, batched=True)

In [9]:
def compute_metrics(eval_preds):
  preds, labels = eval_preds
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels.tolist(), skip_special_tokens=True)
  decoded_preds = tokenizer.batch_decode(preds.tolist(), skip_special_tokens=True)
  bleu_metric = evaluate.load("bleu")
  references = [[reference_text] for reference_text in decoded_labels]
  bleu_scores = bleu_metric.compute(references=references, predictions=decoded_preds)
  bleu_score_1 = None
  bleu_score_2 = None
  bleu_score_3 = None
  bleu_score_4 = None
  bleu_score_avg = None
  for k, v in bleu_scores.items():
    if k == "precisions":
      bleu_score_1 = v[0]
      bleu_score_2 = v[1]        
      bleu_score_3 = v[2]        
      bleu_score_4 = v[3]
      bleu_score_avg = (bleu_score_1 + bleu_score_2 + bleu_score_3 + bleu_score_4)/4
      break
  return {
    'bleu@1': bleu_score_1,
    'bleu@2': bleu_score_2,
    'bleu@3': bleu_score_3,
    'bleu@4': bleu_score_4,
    'bleu@avg': bleu_score_avg
  }

In [9]:
def compute_metrics(eval_preds):
  preds, labels = eval_preds
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  rouge_metric = evaluate.load("rouge")
  rouge_scores = rouge_metric.compute(references=decoded_labels, predictions=decoded_preds, use_stemmer=True, rouge_types=['rouge1', 'rouge2', 'rougeL'])
  return {k: round(v, 4) for k, v in rouge_scores.items()}

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map='auto')

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="pt")

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name.replace('/', '_').replace('-', '_')}_model_summarization",
    learning_rate=1e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    # auto_find_batch_size=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    predict_with_generate=True,
    group_by_length=True,
    push_to_hub=False,
    save_total_limit=3,
    report_to='none',
    run_name=f'{model_name}',
    save_strategy='epoch',
    evaluation_strategy='no'
)

In [14]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_new_data["train"],
    eval_dataset=tokenized_new_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
torch.cuda.empty_cache()
gc.collect()

40

In [ ]:
trainer.train()

In [16]:
torch.cuda.empty_cache()
gc.collect()
trainer.evaluate()

{'eval_loss': 0.5341997146606445, 'eval_rouge1': 0.3635, 'eval_rouge2': 0.1837, 'eval_rougeL': 0.2778, 'eval_runtime': 1379.3407, 'eval_samples_per_second': 1.595, 'eval_steps_per_second': 0.399, 'epoch': 5.0}


{'eval_loss': 0.5341997146606445,
 'eval_rouge1': 0.3635,
 'eval_rouge2': 0.1837,
 'eval_rougeL': 0.2778,
 'eval_runtime': 1379.3407,
 'eval_samples_per_second': 1.595,
 'eval_steps_per_second': 0.399,
 'epoch': 5.0}

# Test Model Summarization

In [4]:
test_data = pd.read_csv('../../SummarizationVietnamese_Backup/dataset/data_2/full_test_data_summarization.csv')

In [3]:
peft_model_id = './VietAI_vit5_base_model_summarization/checkpoint-21010/'
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, load_in_8bit=True, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(36096, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(36096, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear8bitLt(
                    (base_layer): Linear8bitLt(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
  

In [5]:
references = []
predictions = []
def generate_text():
  for batch_1, batch_2 in tqdm(zip(torch.utils.data.DataLoader(test_data['context'], batch_size=16, shuffle=False), torch.utils.data.DataLoader(test_data['summarization'], batch_size=16, shuffle=False), strict=True), total=int(round(len(test_data)/16, 0))):
    encodings = tokenizer(batch_1, max_length=1024, truncation=True, padding=True, return_tensors="pt").to(device)
    outputs = model.generate(
      **encodings,
      max_length=768
    )
    with tokenizer.as_target_tokenizer():
      references.extend(batch_2)
      predictions.extend([tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=False) for output in outputs])
    torch.cuda.empty_cache()

In [6]:
generate_text()

641it [18:54,  1.77s/it]                         


In [ ]:
test_data[f'generate_vit5'] = test_data['context'].progress_apply(lambda x: generate_text(x))

In [7]:
test_data[f'generate_vit5'] = predictions

In [8]:
test_data

,context,summarization,generate_vit5
0,AQI đo được tại 10 trạm quan trắc trên địa bàn...,AQI đo được tại 10 trạm quan trắc trên địa bàn...,"Kết thúc ngày cuối cùng năm 2018 , chất lượng ..."
1,"Theo đó, Cục An toàn thực phẩm đề nghị Sở Y tế...",Cục An toàn thực phẩm đề nghị Sở Y tế tỉnh Điệ...,Cục An toàn thực phẩm ( Bộ Y tế ) vừa có công ...
2,"Ba ngày qua, do ảnh hưởng triều cường biển Đôn...",Do ảnh hưởng của triều cường và mưa to liên ti...,Mưa lớn trong vài ngày qua khiến mực nước sông...
3,"Cơ quan bảo vệ môi trường Mỹ (EPA, cơ quan rất...","EPA, cơ quan bảo vệ môi trường Mỹ, đánh giá mứ...",Để so sánh mức độ tiết kiệm nhiên liệu giữa xe...
4,Thủ tướng Chính phủ giao Bộ Giáo dục và Đào tạ...,Thủ tướng Chính phủ đã giao Bộ Giáo dục và Đào...,Thủ tướng Chính phủ vừa có ý kiến về việc thàn...
...,...,...,...
10238,Danh sách Phó chủ tịch và Thư ký cho 28 Hội đồ...,Bộ trưởng Bộ Giáo dục & Đào tạo (GD&ĐT) Nguyễn...,"Trong 28 hội đồng giáo sư vừa được bổ nhiệm , ..."
10239,"Ngày 9/7, Công an quận Tân Bình đang phối hợp ...","Kiểm tra căn phòng trọ ở quận Tân Bình, TP.HCM...","Không thấy đôi nam nữ mở cửa như thường ngày ,..."
10240,"Ngày 31/8, Công an thị xã Tân Châu, tỉnh An Gi...","Lôi Thiện Tâm (37 tuổi, ở tỉnh An Giang) thừa ...","Tâm mua ma tuý của người lạ , sau đó mang về c..."
10241,"Sáng 18/7, ông Đinh Viết Cường, Chủ tịch UBND ...","Sau lời qua tiếng lại, Trần Văn L.đã dùng rựa ...","Sau khi xảy ra mâu thuẫn , người mẹ đã dùng rự..."


In [5]:
test_data = pd.read_csv('./VietAI_vit5_base_model_summarization/test_vit5.csv')

In [10]:
test_data.to_csv('./VietAI_vit5_base_model_summarization/test_vit5.csv', index=False)

In [9]:
rouge_metric = evaluate.load("rouge")
rouge_scores = rouge_metric.compute(references=test_data['summarization'].tolist(), predictions=test_data['generate_vit5'].tolist(), use_stemmer=True, rouge_types=['rouge1', 'rouge2', 'rougeL'])

In [10]:
rouge_scores

{'rouge1': 0.3738256646802862,
 'rouge2': 0.18130112536171702,
 'rougeL': 0.25851255103594906}